In [11]:
import tensorflow as tf

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

In [3]:
inp = pd.read_csv('/content/drive/MyDrive/research/input.csv', sep=';', index_col=0)
out = pd.read_csv('/content/drive/MyDrive/research/labels.csv', sep=';', index_col=0)

In [8]:
inp.shape

(8, 39)

In [5]:
out.head(3)

,pos_1,pos_2,pos_3,pos_4,pos_5,pos_6,pos_7,pos_8,pos_9,pos_10,pos_11,pos_12,pos_13,pos_14,pos_15,pos_16,pos_17
case 1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
case 2,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0
case 3,0,0,0,0,0,0,0,1,0,0,1,1,0,0,1,0,0


In [6]:
inp = inp.to_numpy()
out = out.to_numpy()

# inp = tf.convert_to_tensor(inp, dtype=tf.float32)
# out = tf.convert_to_tensor(out, dtype=tf.int32)

In [20]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RepeatedKFold

from keras.models import Sequential
from keras.layers import Dense

In [30]:
# get the model
def get_model(n_inputs, n_outputs):
    model = Sequential()
    model.add(Dense(39, input_dim=n_inputs, kernel_initializer='he_uniform'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(55, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(n_outputs, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam')
    return model

# evaluate a model using repeated k-fold cross-validation
def evaluate_model(X, y):
	results = list()
	n_inputs, n_outputs = X.shape[1], y.shape[1]
	# define evaluation procedure
	cv = RepeatedKFold(n_splits=2, n_repeats=3, random_state=1)
	# enumerate folds
	for train_ix, test_ix in cv.split(X):
		# prepare data
		X_train, X_test = X[train_ix], X[test_ix]
		y_train, y_test = y[train_ix], y[test_ix]
		# define model
		model = get_model(n_inputs, n_outputs)
		# fit model
		model.fit(X_train, y_train, verbose=0, epochs=100)
		# make a prediction on the test set
		yhat = model.predict(X_test)
		# round probabilities to class labels
		yhat = yhat.round()
		# calculate accuracy
		acc = accuracy_score(y_test, yhat)
		# store result
		print('> %.3f' % acc)
		results.append(acc)
	return results

In [31]:
# load dataset
X, y = inp, out
# evaluate model
results = evaluate_model(X, y)
# summarize performance
print('Accuracy: %.3f (%.3f)' % (np.mean(results), np.std(results)))

> 0.000
> 0.000
> 0.000
> 0.000
> 0.250
> 0.000
Accuracy: 0.042 (0.093)
